<a href="https://colab.research.google.com/github/NischalSuresh/mentalHealth_chatbot/blob/main/chatbot_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries

In [6]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import necesssary packages

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

## Load the dataset Counsel_chat from huggingface (https://github.com/nbertagnolli/counsel-chat)

In [34]:
dataset = load_dataset("nbertagnolli/counsel-chat")
dataset = dataset['train'].train_test_split(test_size=0.1)

Repo card metadata block was not found. Setting CardData to empty.


In [35]:
dataset

DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 2497
    })
    test: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 278
    })
})

In [36]:
len(dataset['train'])

2497

In [41]:
dataset['train']['answerText'][:3]

["Anger is a normal emotion, and yet it is a really hard one... how much is too much? and how do you express it without damaging those around you? yes mom's get angry, sometimes too easy... but its their responsibility to manage their anger-- get their own support and put themselves in time outs when they cannot handle it well.\xa0 Since you are the kids, you shouldn't have to fix or calm down mom's anger. Maybe in a calmer time when you guys are being silly talk with her one on one? Tell her\xa0 you are feeling sad and hurt that she gets upset at you girls. Ask her if there is 1 thing you can work on this next month to help her. Ask her if she would work on 1 thing for you this month-- not yelling? taking a time out? telling you before she blows her top so you can exit the room? Also for you, check out a library book on anger so you know how to handle yours, and one on parents and anger. Number one to remember, if you don't feel safe, if she is calling you names, or hurting you, break

In [45]:
print(dataset['train']['answerText'][0])

Anger is a normal emotion, and yet it is a really hard one... how much is too much? and how do you express it without damaging those around you? yes mom's get angry, sometimes too easy... but its their responsibility to manage their anger-- get their own support and put themselves in time outs when they cannot handle it well.  Since you are the kids, you shouldn't have to fix or calm down mom's anger. Maybe in a calmer time when you guys are being silly talk with her one on one? Tell her  you are feeling sad and hurt that she gets upset at you girls. Ask her if there is 1 thing you can work on this next month to help her. Ask her if she would work on 1 thing for you this month-- not yelling? taking a time out? telling you before she blows her top so you can exit the room? Also for you, check out a library book on anger so you know how to handle yours, and one on parents and anger. Number one to remember, if you don't feel safe, if she is calling you names, or hurting you, breaking thin

## Prompt template used in Llama-2 original model :
[INST] \<\<SYS\>\>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
\<\</SYS\>\>
{prompt}[/INST] <br>
 Source:
 - https://huggingface.co/TheBloke/Llama-2-70B-Chat-GGML
 - https://github.com/huggingface/blog/blob/main/llama2.md#fine-tuning-with-peft

 Refer chat_completion fn from generation.py for finetuning template - https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L45

## Format the text prompt to required format

In [16]:
def formatting_prompts_func(counsel_data):
  output_texts = []
  for i in range(len(counsel_data)):
    if(counsel_data['answerText'][i]) is None:
      pass
    else:
      question = (counsel_data['questionText'][i]) if (counsel_data['questionText'][i]) is not None else (counsel_data['questionTitle'][i])
      text = f"</s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\n### Question: {question.strip()}\n### Answer: {(counsel_data['answerText'][i]).strip()}</s>"
      output_texts.append(text)
  return output_texts

## Check data for error (missing entries)

In [17]:
print(len(dataset['train']))
for i in range(len(dataset['train'])):
  try:
    question = (dataset['train']['questionText'][i]) if (dataset['train']['questionText'][i]) is not None else (dataset['train']['questionTitle'][i])
    answer = (dataset['train']['answerText'][i])
  except:
    print("error",i)


2497


In [18]:
out = formatting_prompts_func(dataset['train'])

## check data output

In [48]:
print(out[9])

</s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
### Question: It takes me a long time to fall asleep; I’d estimate about two hours. I often have nightmares, starting with being eaten by a monster, and I often wake up frightened and unable to breathe. I believe I started losing sleep after breaking up with a girlfriend of 8 years. Also, my father’s business went bankrupt and my mother has a chronic condition. I was under a lot of stress, and life lost meaning. After improving my life and developing a habit of running and exercise to release stress, my symptoms improved. However in the past couple months, I started losing sleep again and having nightmares without warning. How can I solve this issue?
### Answer: First of all, exercise is always beneficial for your physical and emotional health. That's great that you have added this in your routine. Have you also considered altering your diet? Certain foods are linked with p

## checking Llama format from llama-recipes

In [ ]:
!pip install llama-recipes


In [ ]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

temp_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

In [ ]:
temp_dataset

In [58]:
tokenizer.decode(temp_dataset[1]['input_ids'])

': I can pick you up on the way to the restaurant.\r\nAgnes: That\'s really kind of you.\r\nMatt: No problem.\r\nAgnes: See you on saturday.\r\nMatt: Yes, looking forward to it.\r\nAgnes: Me too.\n---\nSummary:\nMatt invites Agnes for a date to get to know each other better. They\'ll go to the Georgian restaurant in Kazimierz on Saturday at 6 pm, and he\'ll pick her up on the way to the place.</s></s>Summarize this dialog:\nLucas: Hey! How was your day?\r\nDemi: Hey there! \r\nDemi: It was pretty fine, actually, thank you!\r\nDemi: I just got promoted! :D\r\nLucas: Whoa! Great news!\r\nLucas: Congratulations!\r\nLucas: Such a success has to be celebrated.\r\nDemi: I agree! :D\r\nDemi: Tonight at Death & Co.?\r\nLucas: Sure!\r\nLucas: See you there at 10pm?\r\nDemi: Yeah! See you there! :D\n---\nSummary:\nDemi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.</s></s>Summarize this dialog:\nMark: I just shipped the goods\r\nMark: Tomorrow I’ll send you the tracking